In [56]:
! pip install datasets
! pip install transformers
! pip install sentencepiece
! pip install rouge_score
! pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from datasets import load_dataset
import datasets
from tabulate import tabulate
import nltk
from datetime import datetime
import numpy as np 

In [3]:
from transformers import AutoTokenizer
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

import torch 
model_name = "facebook/bart-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenization
encoder_max_length = 256  # demo
decoder_max_length = 64

In [61]:
data = load_dataset("gigaword",split = 'train[:200]')
data

Dataset({
    features: ['document', 'summary'],
    num_rows: 200
})

In [49]:
train_data, test_data = data.train_test_split(test_size=0.1).values()

In [59]:
from datasets import Dataset
dataset = datasets.DatasetDict({'train':train_data,'validation':test_data})

In [60]:
dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 180
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 20
    })
})

In [62]:
max_input_length = 512
max_target_length = 30

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["document"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["summary"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [64]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [65]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 180
    })
    validation: Dataset({
        features: ['document', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 20
    })
})

In [66]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 8
# Show the training loss with every epoch
logging_steps = len(tokenized_datasets["train"]) // batch_size

args = Seq2SeqTrainingArguments(
    output_dir="finetuned-bart-gigaword",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    push_to_hub=False,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [80]:
import numpy as np
from nltk.tokenize import sent_tokenize
import evaluate

rouge_score = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [37]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [70]:
tokenized_datasets = tokenized_datasets.remove_columns(
    dataset["train"].column_names
)
tokenized_datasets

ValueError: ignored

In [72]:
features = [tokenized_datasets["train"][i] for i in range(2)]
data_collator(features)

{'input_ids': tensor([[    0,  4182, 28627,  8453,  1236,  7372,   512, 22324,     8,  2654,
          1269,  1236,  3876,   784,   257,   354, 21966,  7638, 16191,   992,
          1115,  5109,   139,  2922,    10,  7041,   544,  3553, 46806,    13,
             5, 22560,  1680,     9,    94,   353,   128,    29,  8968,  2456,
          4900,  2058,  2156,  2292,  1851,   128,    29,  2373,   935,  3213,
            11, 47385,   107,   479,     2],
        [    0,   102,  2247,  8969,  1236,  1168,  5357,  3174, 10209,   260,
            15, 18862, 46836,  2156,  2429,   292,    82,  2156,     8,  9055,
          4391,     9,  1611,    15,    10,  5753,  2946,    11,  3461, 35957,
          5794,   479,     2,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
     

In [82]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [83]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 180
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 184
  Number of trainable parameters = 139420416
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,0.850300,1.948749,55.354500,40.586000,54.554300,54.374900
2,0.575800,2.140282,53.620200,35.779800,52.972600,52.788500
3,0.418700,2.202495,58.400900,40.919900,57.898800,57.674600
4,0.244400,2.175703,58.075800,43.768800,56.367400,56.369300
5,0.228200,2.215386,53.951800,41.760400,53.561100,53.533400
6,0.160600,2.342358,56.543200,43.902700,55.406200,55.541700
7,0.094900,2.345886,58.208500,44.502800,56.342100,56.326400
8,0.113800,2.342993,58.208500,44.502800,56.342100,56.326400


***** Running Evaluation *****
  Num examples = 20
  Batch size = 8
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

***** Running Evaluation *****
  Num example

TrainOutput(global_step=184, training_loss=0.3280475133139154, metrics={'train_runtime': 22.9481, 'train_samples_per_second': 62.75, 'train_steps_per_second': 8.018, 'total_flos': 41597840793600.0, 'train_loss': 0.3280475133139154, 'epoch': 8.0})

In [84]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 20
  Batch size = 8
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}



Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}



{'eval_loss': 2.3429930210113525,
 'eval_rouge1': 58.2085,
 'eval_rouge2': 44.5028,
 'eval_rougeL': 56.3421,
 'eval_rougeLsum': 56.3264,
 'eval_runtime': 0.9778,
 'eval_samples_per_second': 20.454,
 'eval_steps_per_second': 3.068,
 'epoch': 8.0}

In [86]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["document"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str

In [90]:
generate_summary(dataset['validation'][0],model)

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}



(tensor([[    2,     0,   267, 12001,   579,  3994, 29005, 18862,    11,  2242,
            354,     2]], device='cuda:0'), ['jean sarkozy wed in paris'])

In [92]:
dataset['validation'][0]

{'document': "jean sarkozy , youngest son of french president nicolas sarkozy , exchanged wedding vows wednesday with an heir to a french retailing fortune in his father 's political fiefdom , west of paris .",
 'summary': "youngest sarkozy marries in dad 's fiefdom"}

In [94]:
trainer.save_model('finetune-small')

Saving model checkpoint to finetune-small
Configuration saved in finetune-small/config.json
Configuration saved in finetune-small/generation_config.json
Model weights saved in finetune-small/pytorch_model.bin
tokenizer config file saved in finetune-small/tokenizer_config.json
Special tokens file saved in finetune-small/special_tokens_map.json


In [95]:
finetune = AutoModelForSeq2SeqLM.from_pretrained('/content/finetune-small')

loading configuration file /content/finetune-small/config.json
Model config BartConfig {
  "_name_or_path": "/content/finetune-small",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    